## 6.1 处理文本数据
### 6.1.1 单词和字符的one-hot编码
**代码清单 6-1** 单词级的one-hot编码（简单示例）

In [ ]:
import numpy as np

# 初始数据：每个样本是列表的一个元素（本例中的样本是一个句子，但也可以是一整篇文档）
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 构建数据中所有标记的索引
token_index = {}
for sample in samples:
    # 利用split方法对样本进行分词。在实际应用中，还需要从样本中去掉标点和特殊字符
    for word in sample.split():
        if word not in token_index:
            # 为每个唯一单词指定一个唯一索引。注意，没有为索引编号0指定单词
            token_index[word] = len(token_index) + 1

# 对样本进行分词。只考虑每个样本前max_length个单词
max_length = 10
# 将结果存在results中
results = np.zeros(shape=(len(samples), max_length, max(token_index.values()) + 1))

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

In [ ]:
token_index

In [ ]:
results